In [7]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import psutil, os
import h5py

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device('cpu')

print(device)

cpu


In [ ]:
#start with just train
train_df = pd.read_csv("data_train.csv")

In [4]:
#load pretrained BERT
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModel.from_pretrained(model_name, output_hidden_states=True).to(device)
model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [18]:
def tokenize_and_save(dataset, tokenizer, out_file, batch_size=32, max_len=256):
    all_encodings = []
    #make sure all texts are strings
    texts = ["" if t is None else str(t) for t in dataset]

    #write to a file
    with h5py.File(out_file, "w") as f:
        datasets = None
        idx = 0

        for i in tqdm(range(0, len(texts), batch_size), desc="Tokenizing", dynamic_ncols=True):
            batch = texts[i:i+batch_size]
            #h5py works natively with NumPy arrays
            encoding = tokenizer(batch, return_tensors="np",
                              truncation=True, padding="max_length", max_length=max_len)

            if datasets is None:
                datasets = {}
                for k,v in encoding.items():
                  shape = (len(texts), v.shape[1])
                  datasets[k] = f.create_dataset(k, shape, dtype=v.dtype)

            for k,v in encoding.items():
                datasets[k][idx:idx+len(batch)] = v
            idx += len(batch)

In [14]:
tokenize_and_save(dataset = train_df["text"].tolist(),
                  tokenizer=tokenizer,
                  out_file = "train_encodings.h5")

Tokenizing:   0%|          | 0/19401 [00:00<?, ?it/s]

In [15]:
val_df   = pd.read_csv("data_val.csv")
test_df  = pd.read_csv("data_test.csv")

In [16]:
tokenize_and_save(dataset = val_df["text"].tolist(),
                  tokenizer=tokenizer,
                  out_file = "val_encodings.h5")

Tokenizing:   0%|          | 0/2156 [00:00<?, ?it/s]

In [19]:
tokenize_and_save(dataset = test_df["text"].tolist(),
                  tokenizer=tokenizer,
                  out_file = "test_encodings.h5")

Tokenizing:   0%|          | 0/1185 [00:00<?, ?it/s]